# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv', na_values = ['nan', 'NaN', 'na', ' '])
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [3]:
sinasc_raw.groupby('munResNome')[['IDADEMAE', 'IDADEPAI']].mean()

,IDADEMAE,IDADEPAI
munResNome,,
Alta Floresta D'Oeste,25.991826,29.548701
Alto Alegre dos Parecis,24.844156,29.179104
Alto Paraíso,24.960177,28.833333
Alvorada D'Oeste,25.770732,30.757282
Ariquemes,25.607866,32.466667
Buritis,25.559177,30.812500
Cabixi,26.037500,34.333333
Cacaulândia,25.546667,36.333333
Cacoal,26.890102,30.931854


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [4]:
# Convertendo a coluna 'DTNASC' para string para poder usar o slice
sinasc_raw['DTNASC'] = sinasc_raw['DTNASC'].astype(str)

# Extrair o dia e o mês da data de nascimento
sinasc_raw['DIA_NASC'] = sinasc_raw['DTNASC'].str.slice(8, 10)  # Extrai o dia
sinasc_raw['MES_NASC'] = sinasc_raw['DTNASC'].str.slice(5, 7)  # Extrai o mês

# Filtrando os dados para o dia de aniversário específico (18/01)
aniversario = sinasc_raw[(sinasc_raw['DIA_NASC'] == '18') & (sinasc_raw['MES_NASC'] == '01')]

# Calculando o peso médio dos bebês por sexo e faixa de escolaridade da mãe
peso_medio = aniversario.groupby(['SEXO', 'ESCMAE'])[['PESO']].mean().reset_index()
peso_medio

,SEXO,ESCMAE,PESO
0,Feminino,1 a 3 anos,2890.000000
1,Feminino,12 anos ou mais,3236.428571
2,Feminino,4 a 7 anos,3247.500000
3,Feminino,8 a 11 anos,3328.695652
4,Masculino,12 anos ou mais,3461.153846
5,Masculino,4 a 7 anos,3331.166667
6,Masculino,8 a 11 anos,3300.250000


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [5]:
# Filtrando os dados para o ano de 2019
ano_2019 = sinasc_raw[sinasc_raw['DTNASC'].str.slice(0, 4) == '2019']

# Agrupando por município e contando o número de bebês nascidos em cada município
grupo_municipio = ano_2019.groupby('munResNome').agg({
    'IDADEMAE': ['mean', 'max', 'min'],
    'IDADEPAI': ['mean', 'max', 'min'],
    'CODESTAB': 'count'  # Contando o número de bebês nascidos
})

# Renomeando as colunas para melhorar a interpretação
grupo_municipio.columns = ['idade_media_maes', 'idade_maxima_maes', 'idade_minima_maes',
                            'idade_media_pais', 'idade_maxima_pais', 'idade_minima_pais',
                            'qtd_bebes_nascidos']

grupo_municipio_ordenado = grupo_municipio.sort_values(by='qtd_bebes_nascidos')
print('Estatistícas da cidade que menos nasceu bebês no mês de março')
grupo_municipio_ordenado.head(n=10)

Estatistícas da cidade que menos nasceu bebês no mês de março


,idade_media_maes,idade_maxima_maes,idade_minima_maes,idade_media_pais,idade_maxima_pais,idade_minima_pais,qtd_bebes_nascidos
munResNome,,,,,,,
Município ignorado - RO,24.000000,24,24,22.000000,22.0,22.0,1
Castanheiras,27.281250,39,17,30.392857,43.0,17.0,32
Pimenteiras do Oeste,25.075000,40,14,35.600000,45.0,25.0,40
Primavera de Rondônia,27.023256,39,16,31.888889,44.0,21.0,43
Parecis,26.454545,41,16,32.473684,61.0,18.0,44
Rio Crespo,26.660000,39,16,28.000000,28.0,28.0,50
São Felipe D'Oeste,26.481481,41,17,31.395833,57.0,18.0,54
Teixeirópolis,26.031250,39,16,31.500000,67.0,20.0,64
Cacaulândia,25.546667,42,16,36.333333,50.0,28.0,75


### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [6]:
# Filtrar os dados para incluir apenas os registros do mês de março
marco_data = sinasc_raw[sinasc_raw['DTNASC'].str.slice(5, 7) == '03']

# Agrupar por município e calcular as estatísticas solicitadas
grupo_municipio_marco = marco_data.groupby('munResNome').agg({
    'QTDFILVIVO': [('média', 'mean'), ('máximo', 'max'), ('mínimo', 'min')],
    'IDADEMAE': [('média', 'mean'), ('máximo', 'max'), ('mínimo', 'min')],
    'IDADEPAI': [('média', 'mean'), ('máximo', 'max'), ('mínimo', 'min')],
    'CODESTAB': [('Quantidade', 'count')]  # Contando o número de bebês nascidos
})
# Ordenar o DataFrame em relação à coluna 'CODESTAB' em ordem decrescente
grupo_municipio_marco_sorted = grupo_municipio_marco.sort_values(by=('CODESTAB', 'Quantidade'), ascending=False)

print('Estatistícas da cidade que mais nasceu bebês no mês de março')
# Mostrar o DataFrame ordenado
grupo_municipio_marco_sorted.head(n=1)



Estatistícas da cidade que mais nasceu bebês no mês de março


QTDFILVIVO                 IDADEMAE                 IDADEPAI  \
                 média máximo mínimo      média máximo mínimo      média   
munResNome                                                                 
Porto Velho   1.295056    9.0    0.0  26.596774     44     14  34.629032   

                            CODESTAB  
            máximo mínimo Quantidade  
munResNome                            
Porto Velho   62.0   19.0        743

### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

É curioso constatar que a idade média dos pais é sempre maior que a idade média das mães.

In [7]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResLon     float64
munResAlt     float64
munResArea    float64
DIA_NASC       object
MES_NASC       object
Length: 71, dtype: object

In [8]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf,DIA_NASC,MES_NASC
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,19,02
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,21,02
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,25,02
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,20,03
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,23,03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia,13,12
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia,04,10
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia,02,08
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia,23,12


In [9]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea',
       'DIA_NASC', 'MES

Porcentagem e quantidade de nascimentos por período de gestação em funcção da escolaridade da mãe

In [10]:
# Calcular a contagem de cada gestação por grupo de escolaridade
contagem_gestacao = sinasc_raw.groupby(['ESCMAE', 'GESTACAO']).size().reset_index()

# Calcular a porcentagem de cada período de gestação por grupo de escolaridade
porcentagem_gestacao = sinasc_raw.groupby('ESCMAE')['GESTACAO'].value_counts(normalize=True) * 100

porcentagem_gestacao = porcentagem_gestacao.reset_index()

# Mesclar os DataFrames de contagem e porcentagem
resultado_final = pd.merge(porcentagem_gestacao, contagem_gestacao, on=['ESCMAE', 'GESTACAO'])

resultado_final.columns = ['Eslaridade mãe', 'Tempo de gestação', 'Porcentagem', 'Quantidade']

resultado_final


,Eslaridade mãe,Tempo de gestação,Porcentagem,Quantidade
0,1 a 3 anos,37 a 41 semanas,82.653061,405
1,1 a 3 anos,32 a 36 semanas,12.244898,60
2,1 a 3 anos,42 semanas e mais,3.265306,16
3,1 a 3 anos,28 a 31 semanas,1.020408,5
4,1 a 3 anos,22 a 27 semanas,0.816327,4
5,12 anos ou mais,37 a 41 semanas,88.480209,4739
6,12 anos ou mais,32 a 36 semanas,8.513816,456
7,12 anos ou mais,42 semanas e mais,1.829724,98
8,12 anos ou mais,28 a 31 semanas,0.821509,44
9,12 anos ou mais,22 a 27 semanas,0.298730,16


Quantidade de nascimentos de meninos nascidos em agosto por tipo de parto por municipio

In [11]:
sinasc_raw['DTNASC'] = sinasc_raw['DTNASC'].astype(str)
sinasc_raw['Mes_Nasc'] = sinasc_raw['DTNASC'].str.slice(5,7)

mes_8_meninos = sinasc_raw[(sinasc_raw['Mes_Nasc'] == '08') & (sinasc_raw['SEXO'] == 'Masculino')]

quantidade_por_mucinipio = mes_8_meninos.groupby('munResNome')['PARTO'].value_counts().reset_index()
procentagem_por_mucinipio = mes_8_meninos.groupby('munResNome')['PARTO'].value_counts(normalize=True) * 100

resultado_final_1 = pd.merge(quantidade_por_mucinipio, procentagem_por_mucinipio, on=['munResNome', 'PARTO'])
resultado_final_1.columns = ['Município', 'Tipo de parto', 'Quantidade', 'Porcentagem']
resultado_final_1['Porcentagem'] = resultado_final_1['Porcentagem'].round(2)

resultado_final_1.sort_values(by=('Município'), ascending=True).tail(n=87)



,Município,Tipo de parto,Quantidade,Porcentagem
1,Alta Floresta D'Oeste,Vaginal,2,16.67
2,Alto Alegre dos Parecis,Cesáreo,8,88.89
3,Alto Alegre dos Parecis,Vaginal,1,11.11
4,Alto Paraíso,Cesáreo,9,64.29
5,Alto Paraíso,Vaginal,5,35.71
...,...,...,...,...
83,Vale do Anari,Cesáreo,1,25.00
84,Vale do Paraíso,Vaginal,1,50.00
85,Vale do Paraíso,Cesáreo,1,50.00
86,Vilhena,Cesáreo,50,74.63


Existe uma ampla preferência pelo parto por cesária

In [12]:
sinasc_raw['DTNASC'] = sinasc_raw['DTNASC'].astype(str)
sinasc_raw['Mes_Nasc'] = sinasc_raw['DTNASC'].str.slice(5,7)

mes_8_meninos = sinasc_raw[(sinasc_raw['Mes_Nasc'] == '08') & (sinasc_raw['SEXO'] == 'Masculino')]

quantidade_por_mucinipio_2 = mes_8_meninos.groupby('ESCMAE')['PARTO'].value_counts().reset_index()
procentagem_por_mucinipio_2 = mes_8_meninos.groupby('ESCMAE')['PARTO'].value_counts(normalize=True)*100

resultado_final_2 = pd.merge(quantidade_por_mucinipio_2, procentagem_por_mucinipio_2, on=['ESCMAE','PARTO'])
resultado_final_2 .columns = ['ESCOMAE', 'Tipo de parto', 'Quantidade', 'Porcentagem']
resultado_final_2



,ESCOMAE,Tipo de parto,Quantidade,Porcentagem
0,1 a 3 anos,Cesáreo,18,62.068966
1,1 a 3 anos,Vaginal,11,37.931034
2,12 anos ou mais,Cesáreo,203,81.200000
3,12 anos ou mais,Vaginal,47,18.800000
4,4 a 7 anos,Cesáreo,111,56.923077
5,4 a 7 anos,Vaginal,84,43.076923
6,8 a 11 anos,Cesáreo,425,67.353407
7,8 a 11 anos,Vaginal,206,32.646593
8,Nenhuma,Cesáreo,2,100.000000


Embora seja uma amostragem menor, por aqui notamos que quanto maior a escolaridade, maior é a preferência pelo parto por cesária

In [13]:

quantidade_por_mucinipio_2 = sinasc_raw.groupby('ESCMAE')['PARTO'].value_counts().reset_index()
procentagem_por_mucinipio_2 = sinasc_raw.groupby('ESCMAE')['PARTO'].value_counts(normalize=True)*100

resultado_final_2 = pd.merge(quantidade_por_mucinipio_2, procentagem_por_mucinipio_2, on=['ESCMAE','PARTO'])
resultado_final_2 .columns = ['ESCOMAE', 'Tipo de parto', 'Quantidade', 'Porcentagem']
resultado_final_2['Porcentagem'] = resultado_final_2['Porcentagem'].round(2)
resultado_final_2


,ESCOMAE,Tipo de parto,Quantidade,Porcentagem
0,1 a 3 anos,Cesáreo,273,53.01
1,1 a 3 anos,Vaginal,242,46.99
2,12 anos ou mais,Cesáreo,4698,81.72
3,12 anos ou mais,Vaginal,1051,18.28
4,4 a 7 anos,Cesáreo,2606,54.79
5,4 a 7 anos,Vaginal,2150,45.21
6,8 a 11 anos,Cesáreo,10208,65.41
7,8 a 11 anos,Vaginal,5399,34.59
8,Nenhuma,Vaginal,21,52.50
9,Nenhuma,Cesáreo,19,47.50


Agora está bem claro, usando todo o espaço amostral da base de dados, que quanto maior a escolaridade da mão, maior a preferência por

Relação Apgar com tipo de parto

In [14]:
sinasc_raw.loc[sinasc_raw ['APGAR5'] >= 8, 'APGAR5_cat'] = 'Normal'
sinasc_raw.loc[(sinasc_raw['APGAR5'] >= 6) & (sinasc_raw['APGAR5'] <= 7), 'APGAR5_cat'] = 'Asfixia Leve'
sinasc_raw.loc[(sinasc_raw['APGAR5'] >= 4) & (sinasc_raw['APGAR5'] <= 5), 'APGAR5_cat'] = 'Asfixia Moderada'
sinasc_raw.loc[sinasc_raw['APGAR5'] <= 3, 'APGAR5_cat'] = 'Asfixia Severa'
sinasc_raw['APGAR5_cat'].replace('nan', 'Normal', inplace=True)

In [15]:
apgar5_parto = sinasc_raw.groupby('PARTO')['APGAR5_cat']
apgar5_parto_contagem = apgar5_parto.value_counts().reset_index()
apgar5_parto_porcentagem = apgar5_parto.value_counts(normalize=True)*100

apgar5_parto_df = pd.merge(apgar5_parto_contagem, apgar5_parto_porcentagem, on= ['PARTO', 'APGAR5_cat'])
apgar5_parto_df.columns = ['Tipo de parto', 'Apagar 5', 'Quantidade', 'Proporção']

apgar5_parto_df

,Tipo de parto,Apagar 5,Quantidade,Proporção
0,Cesáreo,Normal,17764,98.491905
1,Cesáreo,Asfixia Leve,200,1.108893
2,Cesáreo,Asfixia Moderada,43,0.238412
3,Cesáreo,Asfixia Severa,29,0.160790
4,Vaginal,Normal,8754,97.886615
5,Vaginal,Asfixia Leve,120,1.341832
6,Vaginal,Asfixia Severa,44,0.492005
7,Vaginal,Asfixia Moderada,25,0.279548


Nota-se uma leve diferença no apgar 5 em relação ao tipo de parto realizado. Por cesária tem uma têndencia um pouco maior de se ter um apgar5 normal do que por parto normal. Mas isso tambem pode estar associado a escolaridade da mãe, ja que quanto menor a escolaridade maior é a frequência de partos normais.